In [ ]:
!pip install pydicom

In [28]:
import numpy as np
import pydicom
import pandas as pd
import os
from keras_preprocessing.image import img_to_array
from tqdm import tqdm
import cv2 as cv
from keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.applications.densenet import DenseNet121
from keras.layers import Dropout, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.models import Model
from sklearn.model_selection import train_test_split

Change the path to where the file train_labels.csv located.

In [29]:
train_df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_labels.csv")

Load .dcm  format image and change the value pixels betwwen 0- 255.
Retrun the proccesed image. 

In [30]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8) #make pixel values between 0-255
    return data


Scan the folder_path to find all the mri_filter data, load and resize the image.
Add the image to data_list and the label to label_list. Return the images of a specific filter and the labels of the images.



In [31]:
def load_and_preprocess(folder_path, mri_filter):
    data_list = []
    label_list = []
    for i in tqdm(range(len(train_df))):  # tqdm suppose to be faster and it shows a status bar
        idt = train_df.loc[i, 'BraTS21ID']  # the value in BraTS21ID column
        idt2 = ('00000' + str(idt))[-5:]  # create folder name 00000 + value in BraTS21ID column.
        path = os.path.join(folder_path, idt2, mri_filter)  # create the path to the filter folder
        for im in os.listdir(path):  # os.listdir(path)= all files in the path
            img = load_dicom(os.path.join(path, im))  # open image in the path
            img = cv.resize(img, (128, 128)) #resize the image
            image = img_to_array(img)  
            data_list+=[image] #add image to data list
            label_list += [train_df.loc[i, 'MGMT_value']]  # add label of image. the same value for all folder
    return np.array(data_list), np.array(label_list) #return the images,labels

Used so the connection to google colab will last longer.

In [32]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

Load and process the data(images and lables)
X will contain the images and y will contain the labels of those images.

In [ ]:
train_dir=r'/content/drive/MyDrive/Colab Notebooks/T2w'
X, y = load_and_preprocess(train_dir, "T2w")

Split the data to 80% training 15% validation and 5% testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test,y_val, y_test = train_test_split(X_test ,y_test, test_size=0.25, random_state=42)


Plot the training and validation accuracy grpah.

In [ ]:
def plot_accuracy(hist):
    get_ac_train = hist.history['accuracy']
    get_ac_val = hist.history['val_accuracy']
    epochs = range(len(get_ac_train))
    plt.plot(epochs, get_ac_train, 'g', label='Accuracy of training data')
    plt.plot(epochs, get_ac_val, 'r', label='Accuracy of validation data')
    plt.title('Accuracy')
    plt.legend(loc=0)
    plt.figure()
    plt.show()


Plot the training and validation loss grpah.




In [ ]:
def plot_loss(hist):
    get_los_train = hist.history['loss']
    get_los_val = hist.history['val_loss']
    epochs = range(len(get_los_train))
    plt.plot(epochs, get_los_train, 'g', label='Loss of training data')
    plt.plot(epochs, get_los_val, 'r', label='Loss of Validation data')
    plt.title('Loss')
    plt.legend(loc=0)
    plt.figure()
    plt.show()


Predict the testing data.
Use the prediction and the labels to calculate the TP, FP, TN,FN.
Calcaute and print the accuracy, precision and reacll using the TP, FP, TN,FN.

In [ ]:
def predict_test_data(X_test):
    y_pred = dense_net.predict(X_test)

    TP1 = 0
    FP1 = 0
    TN1 = 0
    FN1 = 0

    for i in range(len(X_test)): 
      if y_test[i]==1 and y_pred[i][1] >= y_pred[i][0]:
        TP1 += 1
      if y_test[i]==0 and y_pred[i][1] > y_pred[i][0]:
        FP1 += 1
      if y_test[i]==0 and y_pred[i][0] >= y_pred[i][1]:
        TN1 += 1
      if y_test[i]==1 and y_pred[i][0] >= y_pred[i][1]:
        FN1 += 1
    
    print(f"TP = {TP1}, FP = {FP1}, TN = {TN1}, FN = {FN1}")

    if TP1 >0 or (FP1 >0 and FN1>0): #make sure to not divide by zero
      print(f"Accuracy = {(TP1+TN1)/(TP1+FP1+TN1+FN1)}")
      print(f"Precision = {TP1/(TP1+FP1)}")
      print(f"Recall = {TP1/(TP1+FN1)}")




Create the Densenet121 model with random weights initialization, without the top and with input shape width = 128 height = 128 and channels = 1, output shape is 2.
Add layers to the model, compile.
Return the compiled model.

In [ ]:
def create_dense_net():
    model_d=DenseNet121(weights=None,include_top=False, input_shape=(128, 128, 1)) 

    x=model_d.output

    x= GlobalAveragePooling2D()(x)
    x= BatchNormalization()(x)
    x= Dropout(0.5)(x)
    x= Dense(1024,activation='relu')(x) 
    x= Dense(512,activation='relu')(x) 
    x= BatchNormalization()(x)
    x= Dropout(0.5)(x)

    preds=Dense(2,activation='softmax')(x) #FC-layer

    model = Model(inputs=model_d.input, outputs=preds)
    opt = keras.optimizers.Adam(learning_rate=0.0005)#change the ADAM optimizer learning rate
    model.compile(optimizer=opt,loss = keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])#compile the model with ADAM optimizer, SparseCategoricalCrossentropy loss and accuracy metrics

    return model



Create the Densenet121 model.
Train the Densenet121 model with the training and validation data.
Save model weights.
Plot the accuracy graph.
Plot the loss graph.
Predict the testing data and print the accuracy, presicon and recall.

In [ ]:
dense_net = create_dense_net()
hist = dense_net.fit(x=X_train, y=y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val))
dense_net.save_weights(r"/content/drive/MyDrive/Colab Notebooks/1/myModel")
plot_accuracy(hist)
plot_loss(hist)
predict_test_data(X_test)